In [1]:
#%display latex
import random

p = 5
m = 2
q = p^m

K = GF(p)
F.<z> = K.extension(m)

## Key generation

In [2]:
n = 16
k = 8
X = random.sample([i for i in F if i != 0], n) #the support of RS code
Csec = codes.GeneralizedReedSolomonCode(X, k)
Qlist = []
for i in range(n//2):
    Qlist.append(
        block_diagonal_matrix(random_matrix(K, m, algorithm='unimodular'), random_matrix(K, m, algorithm='unimodular')) *
        block_diagonal_matrix(random_matrix(K, m-1, algorithm='unimodular'), random_matrix(K, m+1, algorithm='unimodular'))
    )
Q = block_diagonal_matrix(Qlist)

expMat = lambda G: block_matrix([[j.matrix().T for j in i] for i in G]) #returns subfield image of G

S = random_matrix(K, k*m, algorithm='unimodular')
Gpub = S * expMat(Csec.generator_matrix()) * Q
Cpub = codes.LinearCode(Gpub)
Cpub

[32, 16] linear code over GF(5)

In [ ]:
Csec.generator_matrix().rref()

In [3]:
Gsys = Gpub.rref()
Gsys

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 3 3 2 2 1 2 4 1 3 1 4 2 2 0 2]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 4 0 3 1 2 1 0 0 4 0 4 1 2 4]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 4 2 0 0 0 4 1 0 1 1 3 1 2 2]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 1 2 4 4 3 0 1 4 0 4 1 3 2]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 2 0 4 0 1 1 2 1 4 4 4 2 3 2 0 2]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 3 0 4 2 1 0 1 3 2 2 4 2 4 1 4]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 4 3 3 1 1 4 1 2 4 0 2 4 4 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 2 3 4 1 3 4 0 0 0 3 0 1 0 4 1 1]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 3 2 3 4 3 0 0 1 1 3 3 0 4 2 4]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 4 0 2 2 3 2 0 4 0 2 3 3 2 2 2 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 2 1 4 4 0 3 1 3 2 1 3 1 3 2 4 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 2 3 2 4 0 3 0 1 3 1 3 3 4 4 1]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 3 1 1 1 4 1 1 2 2 1 0 3 0 1 2 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 3 3 2 3 3 0 2 3 0 2 0 2 2 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 2 4 1 1 0 4 4 1 4 3 3 2 4 3 2 4]
[0 0 0 0 0

In [4]:
T = lambda i,j: Gsys[(i-1)*2*m:i*2*m, (j-1)*2*m:j*2*m]

Z = vector([z^i for i in range(m)])
def testMat(A):
    """Checks whether A is in Delta$"""
    a = Z.inner_product(A[0][:m])
    b = Z.inner_product(A[0][m:])
    c = Z.inner_product(A[m][:m])
    d = Z.inner_product(A[m][m:])
    return A == block_matrix([[a.matrix().T, b.matrix().T], [c.matrix().T, d.matrix().T]])

## Step 1: Finding $Q_1'$ 

In [5]:
V = lambda i,j,r,s: T(i,j) * (T(r,j)^-1) * T(r,s) * (T(i,s)^-1)

In [6]:
U = []

for j in range(k//2+1, n//2+1):
    for r in range(1, k//2+1):
        if T(r,j).is_singular() or (r==1):
            continue
        for s in range(k//2+1, n//2):
            if T(1,s).is_singular() or (j == s):
                continue
            if V(1, j, r, s) not in U:
                U.append(V(1, j, r, s))  
len(U)

27

In [7]:
[testMat(Qlist[0] * i * Qlist[0]^-1) for i in U]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [ ]:
#using brute-force

Q1s = []
for u in MatrixSpace(K, m):
    if u.is_singular():
        continue
    #d = 0
    for v in MatrixSpace(K, m+1):   
        if v.is_singular():
            continue    
        Q1 = block_diagonal_matrix([u, identity_matrix(m)]) * block_diagonal_matrix([identity_matrix(m-1), v])
        Q1m = Q1^-1
        fl = True
        for Ui in U:
            if not testMat(Q1 * Ui * Q1m):
                fl = False
            break
        if fl:
            if Q1 not in Q1s: Q1s.append(Q1); print(len(Q1s))
            #break
    #if fl:
    #    break
Q1 = Q1s[0]

In [8]:
#probabilistic
while True:
    u = random_matrix(K, m, algorithm='unimodular')
    v = random_matrix(K, m+1, algorithm='unimodular')
    Q1 = block_diagonal_matrix([u, identity_matrix(m)]) * block_diagonal_matrix([identity_matrix(m-1), v])
    Q1m = Q1^-1
    fl = True
    for Ui in U:
        if not testMat(Q1 * Ui * Q1m):
            fl = False
            break
    if fl:
        print("+")
        break  

+


In [ ]:
Qlist_rec = [Q1]*(n//2)
Qlist_rec

## Step 2: Finding $Q_j'$ for $j \geq k/2+1$

In [10]:
W = lambda i,j,r,s: (T(i,j)^-1) * T(i,s) * (T(r,s)^-1) * T(r,j)

In [11]:
j = k//2 + 1
def findQj_prob(j):
    U = []
    B = Q1 * T(1, j)
    for i in range(1, k//2+1):
        if T(i,j).is_singular(): continue
        for r in range(1, k//2+1):
            if i == r: continue
            for s in range(k//2+1, n//2+1):
                if T(r,s).is_singular() or (j == s): continue
                u = W(i,j,r,s)
                if u not in U: U.append(u)
    while True:
        u = random_matrix(K, m, algorithm='unimodular')
        v = random_matrix(K, m+1, algorithm='unimodular')
        Qj = block_diagonal_matrix([u, identity_matrix(m)]) * block_diagonal_matrix([identity_matrix(m-1), v])
        Qjm = Qj^-1
        fl = True
        if not testMat(B * Qjm):
            fl = False
            continue
        for Ui in U:        
            if not testMat(Qj * Ui * Qjm):
                fl = False
                break
        if fl:     
            print('+')
            return Qj
    return None

def findQj_brutforce(j):
    U = []
    B = Q1 * T(1, j)
    for i in range(1, k//2+1):
        if T(i,j).is_singular(): continue
        for r in range(1, k//2+1):
            if i == r: continue
            for s in range(k//2+1, n//2+1):
                if T(r,s).is_singular() or (j == s): continue
                u = W(i,j,r,s)
                if u not in U: U.append(u)
    for u in MatrixSpace(K, m):
        if u.is_singular(): continue
        for v in MatrixSpace(K, m+1):   
            if v.is_singular(): continue    
            Qj = block_diagonal_matrix([u, identity_matrix(m)]) * block_diagonal_matrix([identity_matrix(m-1), v])
            Qjm = Qj^-1
            fl = True
            if not testMat(B * Qjm): continue
            for Ui in U:
                if not testMat(Qj * Ui * Qjm):
                    fl = False
                    break
            if fl:
                print("+")
                return Qj
    return None

In [12]:
for j in range(k//2+1, n//2+1):
    Qlist_rec[j-1] = findQj_prob(j)

+
+
+
+


In [ ]:
Qlist_rec

## Step 3: Finding $Q_i'$ for $2 \leq i \leq k/2$

In [ ]:
i = 2
U = [T(i,j) * (Qlist_rec[j-1]^-1) for j in range(k//2+1, n//2+1)]
U

In [13]:
def findQi_prob(i):
    U = [T(i,j) * (Qlist_rec[j-1]^-1) for j in range(k//2+1, n//2+1)]    
    while True:
        u = random_matrix(K, m, algorithm='unimodular')
        v = random_matrix(K, m+1, algorithm='unimodular')
        Qi = block_diagonal_matrix([u, identity_matrix(m)]) * block_diagonal_matrix([identity_matrix(m-1), v])
        #Qim = Qi^-1
        fl = True
        for Ui in U:        
            if not testMat(Qi*Ui):
                fl = False
                break
        if fl:     
            print('+')
            return Qi
    return None

def findQi_brutforce(i):
    U = [T(i,j) * (Qlist_rec[j-1]^-1) for j in range(k//2+1, n//2+1)]
    for j in MatrixSpace(K, m+1):
        if j.is_singular(): continue
        for i in MatrixSpace(K, m):   
            if i.is_singular(): continue    
            Qi = block_diagonal_matrix([i, identity_matrix(m)]) * block_diagonal_matrix([identity_matrix(m-1), j])
            #Qim = Qi^-1
            fl = True
            for Ui in U:
                if not testMat(Qi*Ui):
                    fl = False
                    break
            if fl:
                print("+")
                return Qi
    return None

In [14]:
for i in range(2, k//2+1):
    Qlist_rec[i-1] = findQi_prob(i)

+
+
+


In [15]:
Qrec = block_diagonal_matrix(Qlist_rec)

In [16]:
def chunks(lst, m):
    """Yield successive m-sized chunks from lst."""
    for i in range(0, len(lst), m):
        yield lst[i:i + m]

def fold_matrix(M):
    """Anti-expansion"""
    return matrix([[F(j) for j in chunks(i, m)] for i in M])

Csec_rec = codes.LinearCode(fold_matrix(Gpub*(Qrec^-1)))
Csec_rec

[16, 8] linear code over GF(25)

## Step 4: Sidelnikov-Shestakov attack

In [17]:
G = Csec_rec.generator_matrix().rref()

In [18]:
a = [0]*n
a[0] = 0
a[1] = z
a[k] = z+1

for v in range(k+1, n):
    i = 0; j = 1; u = k
    coeff = G[i,u]*G[j,v]/(G[j,u]*G[i,v])
    for alpha in F:        
        if (alpha not in a) and coeff == (a[u]-a[j])*(alpha-a[i])/((a[u]-a[i])*(alpha-a[j])):
            a[v] = alpha
            break
for j in range(2, k):
    i = 0; u = k; v = k+1
    coeff = G[i,u]*G[j,v]/(G[j,u]*G[i,v])
    for alpha in F:        
        if (alpha not in a) and coeff == (a[u]-alpha)*(a[v]-a[i])/((a[u]-a[i])*(a[v]-alpha)):
            a[j] = alpha
            break

In [19]:
a

[0,
 z,
 2*z + 4,
 4*z + 2,
 3*z + 1,
 2*z,
 2*z + 3,
 2,
 z + 1,
 2*z + 1,
 z + 4,
 2*z + 2,
 4*z + 3,
 4*z + 4,
 4*z + 1,
 1]

In [20]:
In = identity_matrix(F, n)
T = matrix([(codes.GeneralizedReedSolomonCode(a, k).generator_matrix() * 
      diagonal_matrix(i) * Csec_rec.parity_check_matrix().T).list() for i in In])
b = T.left_kernel().basis()[0]

In [21]:
Gpub_rec = expMat(codes.GeneralizedReedSolomonCode(a, k, b).generator_matrix()) * Qrec

Gpub.solve_left(Gpub_rec)

[3 3 2 1 1 3 4 0 4 4 3 4 3 0 1 3]
[3 1 2 1 4 1 1 2 3 3 3 2 0 2 2 2]
[4 4 3 3 3 4 4 4 2 3 2 1 0 3 4 3]
[4 1 1 4 2 0 2 0 4 1 1 2 1 3 3 2]
[2 2 3 0 3 2 3 0 1 4 2 1 1 1 1 1]
[3 0 3 4 3 3 4 4 2 0 0 3 3 3 0 3]
[3 3 4 2 3 2 0 3 2 1 1 4 0 1 1 4]
[2 1 0 4 4 4 1 2 0 0 1 3 4 0 1 2]
[1 3 4 4 0 1 4 1 1 0 3 3 1 0 2 1]
[0 3 2 2 3 4 2 4 3 2 1 4 4 3 3 0]
[0 4 2 3 2 2 0 2 4 4 4 3 0 4 2 3]
[3 0 0 3 3 2 0 2 0 1 1 0 1 3 2 0]
[2 0 2 2 2 3 3 2 4 4 0 2 1 0 2 3]
[3 0 0 3 4 4 4 2 4 1 2 2 1 4 0 4]
[4 2 4 0 2 2 3 3 2 3 0 2 2 4 1 3]
[3 0 4 3 0 0 3 1 4 3 3 2 3 0 0 1]